### Examples: interMap module

In [ ]:
# Creation of a map displaying a random variable on 4 european countries. 
# The numerical values assigned to each of the countries are randomly generated 
# using numpy.random.uniform and saved into a dictionary having the country code 
# as the key. This dict is transformed to a Pandas DataFrame with 4 rows and 
# having ‘iso2code’ and ‘value’ as columns. The graduated legend is build using
# the ‘inverted’ Reds Plotly colorscale (low values are dark red, intermediate 
# values are red, high values are white)
import numpy as np
import pandas as pd
import plotly.express as px
from vois import interMap

countries = ['DE', 'ES', 'FR', 'IT']

# Generate random values and create a dictionary: key=countrycode, value=random in [0.0,100.0]
d = dict(zip(countries, list(np.random.uniform(size=len(countries),low=0.0,high=100.0))))

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(d.items(), columns=['iso2code', 'value'])

m = interMap.countriesMap(df,
                          code_column='iso2code',
                          height='400px',
                          stroke_width=0.5,
                          stroke_selected='yellow',
                          colorlist=px.colors.sequential.Reds[::-1],
                          codes_selected=['IT'])
display(m)

In [ ]:
# Creation of a map displaying a custom geojson. The numerical values assigned to 
# each of the countries are randomly generated using numpy.random.uniform and saved
# into a dictionary having the country code as the key. This dict is transformed
# to a Pandas DataFrame with 4 rows and having ‘iso2code’ and ‘value’ as columns.
# The graduated legend is build using the ‘inverted’ Reds Plotly colorscale (low 
# values are dark red, intermediate values are red, high values are white)
import numpy as np
import pandas as pd
import plotly.express as px
from vois import interMap

countries = ['DE', 'ES', 'FR', 'IT']

# Generate random values and create a dictionary: key=countrycode, value=random in [0.0,100.0]
d = dict(zip(countries, list(np.random.uniform(size=len(countries),low=0.0,high=100.0))))

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(d.items(), columns=['iso2code', 'value'])

m = interMap.geojsonMap(df,
                        './data/ne_50m_admin_0_countries.geojson',
                        'ISO_A2_EH',   # Internal attribute used as key
                        code_column='iso2code',
                        height='400px',
                        stroke_width=1.5,
                        stroke_selected='yellow',
                        colorlist=px.colors.sequential.Reds[::-1],
                        codes_selected=['IT'])
display(m)

In [ ]:
# Load a custom geojson file and create a legend on the area of the provinces
import plotly.express as px
from ipywidgets import Layout
from vois import geojsonUtils, interMap
from jeodpp import inter, imap

geojson = geojsonUtils.geojsonLoadFile('./data/ItalyProvinces.geojson')

# Create a VectorLayer instance from the geojson string
vector = interMap.interGeojsonToVector(geojson)

# Create the legend on graduated values of SHAPE_AREA attribute
vector.legendSet('line', 'stroke-width', '1.0')
vector.legendSet('line', 'stroke',       '#010101')
vector.colorCustom(px.colors.sequential.Viridis)
vector.legendGraduated('SHAPE_AREA', 'quantile', 10)

# Add the VectorLayer to the map
m = imap.Map(basemap=1, layout=Layout(height='400px'))
p = vector.process()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

# Set the identify
vector.parameter("identifyfield", "DEN_PROV")
inter.identifyPopup(m,p)

display(m)

In [ ]:
# Add a custom RASTER file to the map: population of the World
from jeodpp import inter, imap
from ipywidgets import Layout

coll = inter.ImageCollection("SIMPLE")
p = coll.process()

minvalue =  0.0
maxvalue = 30.0
colorSchema = 'Spectral_soft'

p.parameter("file","/eos/jeodpp/data/base/Population/GLOBAL/WorldPop/VER1-0/Data/VRT/MOSAIC_ppp_prj_2020.vrt")
p.parameter("epsg", "4326")
p.parameter("nodata", "-99999.0")
p.parameter("scalemin", str(minvalue))
p.parameter("scalemax", str(maxvalue))
p.parameter("interpolate", 'nearest')  # 'bilinear'
p.parameter("colorscheme", colorSchema)

display(HTML(inter.colorSchemaLegend(colorSchema, Title="Population density per km2 in 2020", Markers=True, Height=32, MinValue=minvalue, MaxValue=maxvalue, NumPixels=700))) 

m = imap.Map(basemap=1, layout=Layout(height='600px'))
m.addLayer(p.toLayer())

inter.identifyPopup(m,p)

display(m)

In [ ]:
# Bivariate choropleth map on italian provinces
from ipywidgets import widgets, Layout
from jeodpp import inter, imap
from vois import interMap, geojsonUtils

geojson = geojsonUtils.geojsonLoadFile('./data/ItalyProvinces.geojson')
vector = interMap.interGeojsonToVector(geojson)
vector = vector.parameter("identifyfield", "SIGLA DEN_PROV SHAPE_AREA")
vector = vector.parameter("identifyseparator", "<br>")

m = imap.Map(basemap=1, layout=Layout(height='400px'))

colorlist1 = ['#f3f3f3', '#eac5dd', '#e6a3d0']
colorlist2 = ['#f3f3f3', '#c2f1d5', '#8be2ae']

svg = interMap.bivariateLegend(vector,
                               ["[SIGLA] < 'FE'", "[SIGLA] >= 'FE' and [SIGLA] <= 'PU'", "[SIGLA] > 'PU'"],
                               ["[SHAPE_AREA] < 2500000000", "[SHAPE_AREA] >= 2500000000 and [SHAPE_AREA] <= 4500000000", "[SHAPE_AREA] > 4500000000 and [SHAPE_AREA] <= 7500000000", '[SHAPE_AREA] > 7500000000'],
                               #["[SHAPE_AREA] < 2566597560", "[SHAPE_AREA] >= 2566597560 and [SHAPE_AREA] <= 5566597560", '[SHAPE_AREA] > 5566597560'],
                               colorlist1,
                               colorlist2,
                               title='Example of Bivariate Choropleth Map',
                               title1="Province initials",
                               names1=['< FE', 'in [FE,PU]', '> PU'],
                               title2="Province area",
                               names2=['Small', 'Medium', 'Large', 'XLarge'],
                               fontsize=18,
                               fontweight=500,
                               side=80)

p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

display(HTML(svg))
display(m)